In [1]:
import pandas as pd
import numpy as np
import os

# Define file paths
data_dir = "D:\Ecommerce dataset"

# Load datasets
customers = pd.read_csv(os.path.join(data_dir, "olist_customers_dataset.csv"))
geolocation = pd.read_csv(os.path.join(data_dir, "olist_geolocation_dataset.csv"))
order_items = pd.read_csv(os.path.join(data_dir, "olist_order_items_dataset.csv"))
order_payments = pd.read_csv(os.path.join(data_dir, "olist_order_payments_dataset.csv"))
order_reviews = pd.read_csv(os.path.join(data_dir, "olist_order_reviews_dataset.csv"))
orders = pd.read_csv(os.path.join(data_dir, "olist_orders_dataset.csv"))
products = pd.read_csv(os.path.join(data_dir, "olist_products_dataset.csv"))
sellers = pd.read_csv(os.path.join(data_dir, "olist_sellers_dataset.csv"))
category_translation = pd.read_csv(os.path.join(data_dir, "product_category_name_translation.csv"))

# Display dataset shapes to understand sizes
{
    "customers": customers.shape,
    "geolocation": geolocation.shape,
    "order_items": order_items.shape,
    "order_payments": order_payments.shape,
    "order_reviews": order_reviews.shape,
    "orders": orders.shape,
    "products": products.shape,
    "sellers": sellers.shape,
    "category_translation": category_translation.shape,
}


<>:6: SyntaxWarning: invalid escape sequence '\E'
<>:6: SyntaxWarning: invalid escape sequence '\E'
C:\Users\Asus\AppData\Local\Temp\ipykernel_18808\90377721.py:6: SyntaxWarning: invalid escape sequence '\E'
  data_dir = "D:\Ecommerce dataset"


{'customers': (99441, 5),
 'geolocation': (1000163, 5),
 'order_items': (112650, 7),
 'order_payments': (103886, 5),
 'order_reviews': (99224, 7),
 'orders': (99441, 8),
 'products': (32951, 9),
 'sellers': (3095, 4),
 'category_translation': (71, 2)}

In [3]:
customers.drop_duplicates(inplace=True)
geolocation.drop_duplicates(inplace=True)
order_items.drop_duplicates(inplace=True)
order_payments.drop_duplicates(inplace=True)
order_reviews.drop_duplicates(inplace=True)
orders.drop_duplicates(inplace=True)
products.drop_duplicates(inplace=True)
sellers.drop_duplicates(inplace=True)
category_translation.drop_duplicates(inplace=True)

In [4]:
if "order_status" in orders.columns:
    orders["order_status"].fillna("Unknown", inplace=True)

# For order_items: fill missing numeric values (price, freight_value) with median
for col in ["price", "freight_value"]:
    if col in order_items.columns:
        median_val = order_items[col].median()
        order_items[col].fillna(median_val, inplace=True)

# For order_reviews: fill missing review comments with "No review"
if "review_comment_message" in order_reviews.columns:
    order_reviews["review_comment_message"].fillna("No review", inplace=True)



C:\Users\Asus\AppData\Local\Temp\ipykernel_18808\3647401825.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  orders["order_status"].fillna("Unknown", inplace=True)
C:\Users\Asus\AppData\Local\Temp\ipykernel_18808\3647401825.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For

In [ ]:
# 4. Convert Data Types (Dates & Numbers)

# Define date columns for orders and reviews
order_date_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]
for col in order_date_cols:
    if col in orders.columns:
        orders[col] = pd.to_datetime(orders[col], errors="coerce")
        
review_date_cols = ["review_creation_date", "review_answer_timestamp"]
for col in review_date_cols:
    if col in order_reviews.columns:
        order_reviews[col] = pd.to_datetime(order_reviews[col], errors="coerce")

# 5. Merge Product Categories

# Merge products with category translation to get English category names
if "product_category_name" in products.columns and "product_category_name" in category_translation.columns:
    products = products.merge(category_translation, on="product_category_name", how="left")
    # Remove the original category column and rename the translated one
    products.drop(columns=["product_category_name"], inplace=True)
    products.rename(columns={"product_category_name_english": "product_category"}, inplace=True)


# 6. Feature Engineering

# a. Order Processing Time (days) = Delivered Customer Date - Purchase Timestamp
if "order_delivered_customer_date" in orders.columns and "order_purchase_timestamp" in orders.columns:
    orders["order_processing_time"] = (orders["order_delivered_customer_date"] - orders["order_purchase_timestamp"]).dt.days

# b. Delivery Delay (days) = Delivered Customer Date - Estimated Delivery Date
if "order_delivered_customer_date" in orders.columns and "order_estimated_delivery_date" in orders.columns:
    orders["delivery_delay"] = (orders["order_delivered_customer_date"] - orders["order_estimated_delivery_date"]).dt.days

# c. Calculate Total Order Value for each order item (price + freight_value)
if set(["price", "freight_value"]).issubset(order_items.columns):
    order_items["total_order_value"] = order_items["price"] + order_items["freight_value"]

# d. Aggregate Revenue per Seller from order items
seller_revenue = order_items.groupby("seller_id", as_index=False)["total_order_value"].sum()
seller_revenue.rename(columns={"total_order_value": "total_seller_revenue"}, inplace=True)

# e. Customer Order Count
customer_orders = orders.groupby("customer_id", as_index=False).size()
customer_orders.rename(columns={"size": "order_count"}, inplace=True)

# 7. Merge Datasets for a Comprehensive Table

# Merge Orders with Order Items (using order_id)
orders_merged = orders.merge(order_items, on="order_id", how="left", suffixes=("", "_item"))

# Merge Orders with Customers (using customer_id)
orders_merged = orders_merged.merge(customers, on="customer_id", how="left", suffixes=("", "_customer"))

# Merge with Seller Revenue (using seller_id)
if "seller_id" in orders_merged.columns:
    orders_merged = orders_merged.merge(seller_revenue, on="seller_id", how="left")

# Optionally, merge Products info (using product_id)
if "product_id" in orders_merged.columns:
    orders_merged = orders_merged.merge(products, on="product_id", how="left", suffixes=("", "_product"))

# Merge Order Reviews (using order_id)
orders_merged = orders_merged.merge(order_reviews, on="order_id", how="left", suffixes=("", "_review"))


# 8.Save the Cleaned and Engineered Data

output_path = os.path.join(data_dir, "cleaned_orders_merged.csv")
orders_merged.to_csv(output_path, index=False)
print("Cleaned and merged dataset saved to:", output_path)


# 9. Display a Sample of the Final DataFrame

orders_merged.head()

Cleaned and merged dataset saved to: D:\Ecommerce dataset\cleaned_orders_merged.csv


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_processing_time,delivery_delay,...,product_length_cm,product_height_cm,product_width_cm,product_category,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8.0,-8.0,...,19.0,8.0,13.0,housewares,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11,2017-10-12 03:43:48
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13.0,-6.0,...,19.0,13.0,19.0,perfumery,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08,2018-08-08 18:37:50
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9.0,-18.0,...,24.0,19.0,21.0,auto,e73b67b67587f7644d5bd1a52deb1b01,5.0,NaN,No review,2018-08-18,2018-08-22 19:07:58
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13.0,-13.0,...,30.0,10.0,20.0,pet_shop,359d03e676b3c069f62cadba8dd3f6e8,5.0,NaN,O produto foi exatamente o que eu esperava e e...,2017-12-03,2017-12-05 19:21:58
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2.0,-10.0,...,51.0,15.0,15.0,stationery,e50934924e227544ba8246aeb3770dd4,5.0,NaN,No review,2018-02-17,2018-02-18 13:02:51


In [10]:
orders_merged.describe()

,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_processing_time,delivery_delay,order_item_id,price,freight_value,...,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,review_score,review_creation_date,review_answer_timestamp
count,114092,113930,112112,110839,114092,110839.000000,110839.000000,113314.000000,113314.000000,113314.000000,...,111702.000000,111702.000000,111702.000000,113296.000000,113296.000000,113296.000000,113296.000000,113131.000000,113131,113131
mean,2017-12-31 03:25:40.890754816,2017-12-31 13:42:12.637856768,2018-01-04 17:16:10.851951360,2018-01-14 04:20:16.402466304,2018-01-23 23:37:13.103109888,12.012414,-12.035809,1.198528,120.478701,19.979428,...,48.777560,786.899250,2.206908,2091.915037,30.162495,16.584513,23.003539,4.016998,2018-01-12 21:52:44.236151296,2018-01-16 01:23:17.639930880
min,2016-09-04 21:15:19,2016-09-15 12:16:38,2016-10-08 10:34:01,2016-10-11 13:46:32,2016-09-30 00:00:00,0.000000,-147.000000,1.000000,0.850000,0.000000,...,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000,1.000000,2016-10-02 00:00:00,2016-10-07 18:32:28
25%,2017-09-12 21:26:59.500000,2017-09-13 12:55:21.249999872,2017-09-18 18:03:40.500000,2017-09-26 17:35:15,2017-10-04 00:00:00,6.000000,-17.000000,1.000000,39.900000,13.080000,...,42.000000,348.000000,1.000000,300.000000,18.000000,8.000000,15.000000,4.000000,2017-09-25 00:00:00,2017-09-27 23:20:49.500000
50%,2018-01-18 20:36:17.500000,2018-01-19 08:32:32,2018-01-24 15:46:46.500000,2018-02-02 17:29:57,2018-02-15 00:00:00,10.000000,-13.000000,1.000000,74.900000,16.260000,...,52.000000,601.000000,1.000000,700.000000,25.000000,13.000000,20.000000,5.000000,2018-02-02 00:00:00,2018-02-05 08:44:22
75%,2018-05-04 00:24:31.249999872,2018-05-04 11:33:19,2018-05-08 11:17:30,2018-05-15 17:55:11,2018-05-25 00:00:00,15.000000,-7.000000,1.000000,134.900000,21.150000,...,57.000000,985.000000,3.000000,1800.000000,38.000000,20.000000,30.000000,5.000000,2018-05-15 00:00:00,2018-05-20 09:59:27.500000
max,2018-10-17 17:30:18,2018-09-03 17:40:06,2018-09-11 19:48:28,2018-10-17 13:22:46,2018-11-12 00:00:00,209.000000,188.000000,21.000000,6735.000000,409.680000,...,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000,5.000000,2018-08-31 00:00:00,2018-10-29 12:27:35
std,NaN,NaN,NaN,NaN,NaN,9.445470,10.159965,0.707016,183.279678,15.783227,...,10.024616,651.758866,1.719500,3749.804597,16.151737,13.439206,11.708481,1.400074,NaN,NaN


In [11]:
orders_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114092 entries, 0 to 114091
Data columns (total 36 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       114092 non-null  object        
 1   customer_id                    114092 non-null  object        
 2   order_status                   114092 non-null  object        
 3   order_purchase_timestamp       114092 non-null  datetime64[ns]
 4   order_approved_at              113930 non-null  datetime64[ns]
 5   order_delivered_carrier_date   112112 non-null  datetime64[ns]
 6   order_delivered_customer_date  110839 non-null  datetime64[ns]
 7   order_estimated_delivery_date  114092 non-null  datetime64[ns]
 8   order_processing_time          110839 non-null  float64       
 9   delivery_delay                 110839 non-null  float64       
 10  order_item_id                  113314 non-null  float64       
 11  

In [17]:
orders_merged


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_processing_time,delivery_delay,...,product_length_cm,product_height_cm,product_width_cm,product_category,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8.0,-8.0,...,19.0,8.0,13.0,housewares,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11,2017-10-12 03:43:48
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13.0,-6.0,...,19.0,13.0,19.0,perfumery,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08,2018-08-08 18:37:50
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9.0,-18.0,...,24.0,19.0,21.0,auto,e73b67b67587f7644d5bd1a52deb1b01,5.0,NaN,No review,2018-08-18,2018-08-22 19:07:58
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13.0,-13.0,...,30.0,10.0,20.0,pet_shop,359d03e676b3c069f62cadba8dd3f6e8,5.0,NaN,O produto foi exatamente o que eu esperava e e...,2017-12-03,2017-12-05 19:21:58
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2.0,-10.0,...,51.0,15.0,15.0,stationery,e50934924e227544ba8246aeb3770dd4,5.0,NaN,No review,2018-02-17,2018-02-18 13:02:51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114087,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,22.0,-2.0,...,40.0,10.0,40.0,baby,29bb71b2760d0f876dfa178a76bc4734,4.0,NaN,So uma peça que veio rachado mas tudo bem rs,2018-03-01,2018-03-02 17:50:01
114088,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,24.0,-6.0,...,32.0,90.0,22.0,home_appliances_2,371579771219f6db2d830d50805977bb,5.0,NaN,Foi entregue antes do prazo.,2017-09-22,2017-09-22 23:10:57
114089,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,17.0,-21.0,...,20.0,20.0,20.0,computers_accessories,8ab6855b9fe9b812cd03a480a25058a1,2.0,NaN,Foi entregue somente 1. Quero saber do outro p...,2018-01-26,2018-01-27 09:16:56
114090,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,17.0,-21.0,...,20.0,20.0,20.0,computers_accessories,8ab6855b9fe9b812cd03a480a25058a1,2.0,NaN,Foi entregue somente 1. Quero saber do outro p...,2018-01-26,2018-01-27 09:16:56
